In [1]:
import boto3
import botocore
import pandas as pd
import os
from dotenv import load_dotenv
from io import BytesIO

In [2]:
load_dotenv()
s3 = boto3.client('s3')
BUCKET_NAME = "half-marathon"

### 1. Wczytanie danych z Digital Ocean Spaces

In [23]:
# Wylistowanie wszystkich plików w Spaces Object Storage
response = s3.list_objects_v2(Bucket=BUCKET_NAME)

for obj in response["Contents"]:
    print(obj["Key"]) 

dane/halfmarathon_wroclaw_2023__final.csv
dane/halfmarathon_wroclaw_2024__final.csv


In [24]:
# Wczytanie danych z Spaces Object Storage
all_dfs = []
response = s3.list_objects_v2(Bucket=BUCKET_NAME)

for obj in response.get("Contents", []):
    file_key = obj["Key"]
    if file_key.endswith('.csv'):
        # Pobieramy dane z chmury
        file_obj = s3.get_object(Bucket=BUCKET_NAME, Key=file_key)
        
        # Tworzymy obiekt BytesIO bezpośrednio z odczytanych bajtów
        bio = BytesIO(file_obj['Body'].read())
        
        # Wczytujemy DataFrame
        df_part = pd.read_csv(bio, sep=';', encoding='utf-8')
        
        # Dodajemy informację o roku/pliku
        df_part['source_file'] = file_key
        all_dfs.append(df_part)

# Łączenie danych w jedną całość
if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Dane wczytane pomyślnie. Kształt ramki danych: {df.shape}")
    # Wyświetlamy pierwsze 5 wierszy, aby upewnić się, że kolumny się rozdzieliły
    display(df.sample(5))

Dane wczytane pomyślnie. Kształt ramki danych: (21957, 28)


,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,source_file
17374,8425.0,6044,MATEUSZ,KOCZOROWSKI,GRUSZCZYN,POL,NaN,M,6299.0,M30,...,01:35:53,7991.0,6.870000,02:13:15,8319.0,7.473333,0.098733,02:22:12,6.740934,dane/halfmarathon_wroclaw_2024__final.csv
20417,NaN,76860,TOMASZ,KUPIEC,NaN,NaN,Wybiegaj. Z. Dupy. Raka,M,NaN,M40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dane/halfmarathon_wroclaw_2024__final.csv
5281,5282.0,7875,GRZEGORZ,SZMOLOWSKI,BYTOM,POL,DREAM TEAM I FUNDACJA DZIĘKI TOBIE,M,4251.0,M40,...,01:30:04,5670.0,5.940000,02:02:09,5334.0,6.416667,0.016333,02:07:52,6.061468,dane/halfmarathon_wroclaw_2023__final.csv
20180,NaN,28271,GRZEGORZ,KILIAN,NaN,NaN,NaN,M,NaN,M50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dane/halfmarathon_wroclaw_2024__final.csv
12444,3495.0,3988,DMITRII,BURASHNIKOV,WROCŁAW,POL,NIE,M,3074.0,M20,...,01:20:36,3661.0,5.553333,01:48:41,3531.0,5.616667,0.024867,01:54:58,5.449949,dane/halfmarathon_wroclaw_2024__final.csv


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21957 entries, 0 to 21956
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Miejsce                    18450 non-null  float64
 1   Numer startowy             21957 non-null  int64  
 2   Imię                       21957 non-null  object 
 3   Nazwisko                   21957 non-null  object 
 4   Miasto                     17774 non-null  object 
 5   Kraj                       18450 non-null  object 
 6   Drużyna                    8402 non-null   object 
 7   Płeć                       21946 non-null  object 
 8   Płeć Miejsce               18450 non-null  float64
 9   Kategoria wiekowa          21926 non-null  object 
 10  Kategoria wiekowa Miejsce  18430 non-null  float64
 11  Rocznik                    21472 non-null  float64
 12  5 km Czas                  18411 non-null  object 
 13  5 km Miejsce Open          18411 non-null  flo

In [26]:
df.isnull().sum()

Miejsce                       3507
Numer startowy                   0
Imię                             0
Nazwisko                         0
Miasto                        4183
Kraj                          3507
Drużyna                      13555
Płeć                            11
Płeć Miejsce                  3507
Kategoria wiekowa               31
Kategoria wiekowa Miejsce     3527
Rocznik                        485
5 km Czas                     3546
5 km Miejsce Open             3546
5 km Tempo                    3546
10 km Czas                    3530
10 km Miejsce Open            3530
10 km Tempo                   3562
15 km Czas                    3529
15 km Miejsce Open            3529
15 km Tempo                   3544
20 km Czas                    3518
20 km Miejsce Open            3518
20 km Tempo                   3535
Tempo Stabilność              3580
Czas                          2055
Tempo                         3507
source_file                      0
dtype: int64

In [27]:
# Łączna liczba wierszy w DataFrame
total_rows = len(df)

# Zliczamy niepuste wartości (posiadane dane) dla każdej kolumny
count_present = df.count()

# Obliczamy % posiadanych danych
percent_present = (count_present / total_rows) * 100

# Obliczamy % brakujących danych
percent_missing = (df.isnull().sum() / total_rows) * 100

# Tabela z wynikami
analysis_df = pd.DataFrame({
    'liczba wierszy': count_present,
    '% posiadanych danych': percent_present.map('{:.2f}%'.format),
    '% brakujących danych': percent_missing.map('{:.2f}%'.format)
})

# Wynik posortowany po liczbie posiadanych danych (od najpełniejszych kolumn)
analysis_df.sort_values(by='liczba wierszy', ascending=False)

,liczba wierszy,% posiadanych danych,% brakujących danych
source_file,21957,100.00%,0.00%
Imię,21957,100.00%,0.00%
Nazwisko,21957,100.00%,0.00%
Numer startowy,21957,100.00%,0.00%
Płeć,21946,99.95%,0.05%
Kategoria wiekowa,21926,99.86%,0.14%
Rocznik,21472,97.79%,2.21%
Czas,19902,90.64%,9.36%
Tempo,18450,84.03%,15.97%
Miejsce,18450,84.03%,15.97%


In [28]:
# Dane wejściowe, które podaje użytkownik do analizy: płeć, wiek, czas na 5km
# Aplikacja ma szacować czas ukończenia półmaratonu na podstawie czasu na 5 km, płci i wieku

summary = analysis_df.loc[['Czas', '5 km Czas', 'Płeć', 'Rocznik', 'Kategoria wiekowa']]
summary.sort_values(by='liczba wierszy', ascending=False)

,liczba wierszy,% posiadanych danych,% brakujących danych
Płeć,21946,99.95%,0.05%
Kategoria wiekowa,21926,99.86%,0.14%
Rocznik,21472,97.79%,2.21%
Czas,19902,90.64%,9.36%
5 km Czas,18411,83.85%,16.15%


### 2. Czyszczenie danych

#### 2.1. Transformacja danych

Kopia DataFrame + przetworzenie kolumny `source_file` na `Rok`.

In [29]:
df_copy = df.copy()

# .str.extract znajduje ciąg cyfr, w tym przypadku 4 cyfr (\d{4})
df_copy['Rok_biegu'] = df_copy['source_file'].str.extract(r'(\d{4})').astype(int)
df_copy = df_copy.drop(columns=['source_file'])

df_copy.sample(5)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Rok_biegu
16097,7148.0,8067,DARIAN,FIDOWICZ,WROCŁAW,POL,-,M,5619.0,M30,...,01:32:42,7350.0,5.886667,02:04:17,6981.0,6.316667,-0.019133,02:12:44,6.292170,2024
16924,7975.0,11158,Anonimowy,ZAWODNIK,NaN,POL,603688001,M,6077.0,M50,...,01:36:29,8128.0,6.336667,02:10:25,7936.0,6.786667,0.000467,02:18:34,6.568697,2024
21541,NaN,76118,JAKUB,TOMCZAK,NaN,NaN,NaN,M,NaN,M30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
18885,9937.0,10759,JERZY,SZAŁAJKO,WROCŁAW,POL,PARKRUN WROCŁAW,M,7070.0,M60,...,01:54:17,10021.0,7.913333,02:35:23,9936.0,8.220000,0.055400,02:45:10,7.829659,2024
16361,7412.0,11198,JACEK,DZIAŁKOWSKI,CHRZĄSTAWA WIELKA,POL,NaN,M,5772.0,M40,...,01:33:48,7582.0,6.496667,02:07:21,7475.0,6.710000,0.038200,02:14:36,6.380659,2024


Obliczenie wieku: `Rok, w którym odbył się bieg` - `Rocznik`

In [30]:
df_copy['Wiek'] = df_copy['Rok_biegu'] - df_copy['Rocznik']

In [31]:
df_copy[['Czas', '5 km Czas', 'Płeć', 'Rocznik', 'Wiek', 'Kategoria wiekowa']]

,Czas,5 km Czas,Płeć,Rocznik,Wiek,Kategoria wiekowa
0,01:04:59,00:14:37,M,1992.0,31.0,M30
1,01:06:23,00:14:48,M,1986.0,37.0,M30
2,01:08:24,00:15:46,M,1996.0,27.0,M20
3,01:10:16,00:16:11,M,1988.0,35.0,M30
4,01:10:27,00:16:12,M,1995.0,28.0,M20
...,...,...,...,...,...,...
21952,DNS,NaN,K,1982.0,42.0,K40
21953,NaN,NaN,K,1998.0,26.0,K20
21954,DNS,NaN,M,1995.0,29.0,M20
21955,NaN,NaN,K,1991.0,33.0,K30


Zamiana czasu na sekundy

In [32]:
def convert_time_to_seconds(time):
    if pd.isnull(time) or time in ['DNS', 'DNF']:
        return None
    time = time.split(':')
    return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])

# Lista kolumn do konwersji
cols_to_convert = ['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas']

for col in cols_to_convert:
    # Używamy astype(str), aby mieć pewność, że split(':') zadziała na tekście
    df_copy[col] = df_copy[col].apply(convert_time_to_seconds)

**Pozostawiam potrzebne kolumny.**

*Kolumny które nie zostały uwzględnione:*

- *Numer startowy*
- *Imię*
- *Nazwisko*
- *Miasto*
- *Kraj*
- *Drużyna*

In [37]:
df_clean = df_copy[[
    'Czas',
    'Wiek',
    'Rocznik',
    'Miejsce',
    'Płeć',
    'Płeć Miejsce',
    'Kategoria wiekowa',
    'Kategoria wiekowa Miejsce',
    '5 km Czas',
    '5 km Miejsce Open',
    '5 km Tempo',
    '10 km Czas',
    '10 km Miejsce Open',
    '10 km Tempo',
    '15 km Czas',
    '15 km Miejsce Open',
    '15 km Tempo',
    '20 km Czas',
    '20 km Miejsce Open',
    '20 km Tempo',
    'Tempo Stabilność',
    'Tempo'
    ]]

df_clean

,Czas,Wiek,Rocznik,Miejsce,Płeć,Płeć Miejsce,Kategoria wiekowa,Kategoria wiekowa Miejsce,5 km Czas,5 km Miejsce Open,...,10 km Miejsce Open,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Tempo
0,3899.0,31.0,1992.0,1.0,M,1.0,M30,1.0,877.0,1.0,...,1.0,2.926667,2687.0,1.0,3.106667,3703.0,1.0,3.386667,0.031400,3.080509
1,3983.0,37.0,1986.0,2.0,M,2.0,M30,2.0,888.0,2.0,...,2.0,2.983333,2726.0,2.0,3.143333,3788.0,2.0,3.540000,0.038000,3.146875
2,4104.0,27.0,1996.0,3.0,M,3.0,M20,1.0,946.0,4.0,...,3.0,3.123333,2854.0,3.0,3.236667,3909.0,3.0,3.516667,0.024067,3.242475
3,4216.0,35.0,1988.0,4.0,M,4.0,M30,3.0,971.0,6.0,...,5.0,3.196667,2929.0,5.0,3.330000,4014.0,4.0,3.616667,0.025467,3.330963
4,4227.0,28.0,1995.0,5.0,M,5.0,M20,2.0,972.0,7.0,...,7.0,3.276667,2971.0,7.0,3.386667,4047.0,5.0,3.586667,0.023000,3.339654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21952,NaN,42.0,1982.0,NaN,K,NaN,K40,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21953,NaN,26.0,1998.0,NaN,K,NaN,K20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21954,NaN,29.0,1995.0,NaN,M,NaN,M20,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21955,NaN,33.0,1991.0,NaN,K,NaN,K30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Czas zawiera wartości NaN, które należy usunąć.

In [38]:
# Usunięcie wierszy z NaN w kolumnie 'Czas'
df_clean = df_clean.dropna(subset=['Czas'])

# Reset indeksu zapewnia ciągłość
df_clean = df_clean.reset_index(drop=True)

print(f"Liczba wierszy po usunięciu braków w kolumnie 'Czas': {len(df_clean)}")
print(f"Braki w kolumnie 'Czas': {df_clean['Czas'].isnull().sum()}")

Liczba wierszy po usunięciu braków w kolumnie 'Czas': 18450
Braki w kolumnie 'Czas': 0


In [39]:
missing_data = df_clean.isnull().sum()
missing_only = missing_data[missing_data > 0]

print("Kolumny z brakującymi danymi:")
missing_only

Kolumny z brakującymi danymi:


Wiek                         485
Rocznik                      485
Kategoria wiekowa             20
Kategoria wiekowa Miejsce     20
5 km Czas                     39
5 km Miejsce Open             39
5 km Tempo                    39
10 km Czas                    23
10 km Miejsce Open            23
10 km Tempo                   55
15 km Czas                    22
15 km Miejsce Open            22
15 km Tempo                   37
20 km Czas                    11
20 km Miejsce Open            11
20 km Tempo                   28
Tempo Stabilność              73
dtype: int64

Wypełniam kolumnę `Wiek` na podstawie kolumny `Kategoria wiekowa`, gdzie wiek przyjmowany jest według wzoru: K40 == 40, M20 == 20

Format danych zmieniony z float na int.

In [40]:
kategorie_num = pd.to_numeric(df_clean['Kategoria wiekowa'].str.extract(r'(\d+)')[0], errors='coerce')

df_clean['Wiek'] = df_clean['Wiek'].fillna(kategorie_num)

if df_clean['Wiek'].isnull().sum() == 0:
    df_clean['Wiek'] = df_clean['Wiek'].astype(int)

print(f"Braki w kolumnie 'Wiek': {df_clean['Wiek'].isnull().sum()}")

Braki w kolumnie 'Wiek': 0


W kolumnie `5 km Czas` pozostało **39** wartości NaN. Usuwam je.

In [41]:
df_clean = df_clean.dropna(subset=['5 km Czas'])
df_clean = df_clean.reset_index(drop=True)

In [42]:
# df z NaN
missing_data = df_clean.isnull().sum()
missing_only = missing_data [missing_data > 0]

if not missing_only.empty:
    summary_df = missing_only.to_frame(name='Liczba braków')
    display(summary_df)
else:
    print("Brak kolumn z wartościami NaN.")

,Liczba braków
Rocznik,484
Kategoria wiekowa,18
Kategoria wiekowa Miejsce,18
10 km Czas,16
10 km Miejsce Open,16
10 km Tempo,16
15 km Czas,16
15 km Miejsce Open,16
15 km Tempo,29
20 km Czas,8


#### 2.2. Zapisanie przygotowanych danych do `.csv`

In [44]:
df_ml = df_clean[[
    'Czas',
    'Wiek',
    # 'Miejsce',
    'Płeć',
    # 'Płeć Miejsce',
    '5 km Czas',
    # '5 km Miejsce Open',
    # '5 km Tempo',
    # '10 km Czas',
    # '10 km Miejsce Open',
    # '10 km Tempo',
    # '15 km Czas',
    # '15 km Miejsce Open',
    # '15 km Tempo',
    # '20 km Czas',
    # '20 km Miejsce Open',
    # '20 km Tempo',
    # 'Tempo Stabilność',
    # 'Tempo'
    ]]

df_ml.sample(5)

,Czas,Wiek,Płeć,5 km Czas
10840,6650.0,24,M,1563.0
18011,9845.0,21,M,2190.0
15567,8094.0,50,K,1857.0
9208,6010.0,44,K,1394.0
6450,8315.0,36,M,1964.0


In [ ]:
# LOKALNIE

folder_path = 'dane/dane_dla_modelu'
file_name = 'halfmarathon_wroclaw_23_24.csv'
full_path = os.path.join(folder_path, file_name)

# Sprawdzam, czy folder istnieje. Jeśli nie, to automatycznie go utworzy
# exist_ok=True zapobiega błędowi, jeśli folder już tam jest
os.makedirs(folder_path, exist_ok=True)

# index=False zapobiega dodawaniu niepotrzebnej kolumny z numerami wierszy
df_ml.to_csv(full_path, index=False, sep=';', encoding='utf-8')

In [3]:
# DO SPACES OBJECT STORAGE

local_directory = 'dane'

for root, dirs, files in os.walk(local_directory):
    for filename in files:
        local_path = os.path.join(root, filename)
        relative_path = os.path.relpath(local_path, os.path.dirname(local_directory))
        s3_path = relative_path.replace("\\", "/")

        try:
            # Sprawdzamy metadane pliku w Spaces (wywołanie head_object)
            # Jeśli plik nie istnieje, boto3 rzuci błąd ClientError (404)
            s3.head_object(Bucket=BUCKET_NAME, Key=s3_path)
            # Jeśli kod przejdzie tutaj, oznacza to, że plik już jest w chmurze
            print(f"Plik {s3_path} już istnieje. Pomijam.")
            
        except botocore.exceptions.ClientError as e:
            # Kod błędu 404 oznacza, że obiektu nie znaleziono w Spaces
            if e.response['Error']['Code'] == "404":
                print(f"Przesyłanie nowej zawartości: {local_path} -> {s3_path}...")
                s3.upload_file(local_path, BUCKET_NAME, s3_path)
            else:
                # Inny błąd (np. brak uprawnień) – rzucamy go dalej
                raise e

Plik dane/halfmarathon_wroclaw_2023__final.csv już istnieje. Pomijam.
Plik dane/halfmarathon_wroclaw_2024__final.csv już istnieje. Pomijam.
Przesyłanie nowej zawartości: dane\dane_dla_modelu\halfmarathon_wroclaw_23_24.csv -> dane/dane_dla_modelu/halfmarathon_wroclaw_23_24.csv...
